In [1]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator
import re

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [451]:
def rank(text,top=0,collapse=True):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    stemmed_words = [stemmer.lemmatize(word) for word in tokens]
    cleaned_words= [word for word in stemmed_words if word not in stops]
    filtered_words=[]
    counts=dict()
    scores=dict()
    for word in cleaned_words:
        filtered=re.sub('[^a-zA-Z]', '', word)
        if(len(filtered)>=3 or filtered in model.wv.vocab):
            filtered_words.append(word)
            counts.setdefault(word,0)
            counts[word]+=1
    scoreList=np.zeros(len(filtered_words))
    for i in range(len(filtered_words)):
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        for j in range(i+1,rightBound+1):
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            similarity_score=(similarity_score+1)/2.0
            scoreList[i]+=similarity_score
            scoreList[j]+=similarity_score
        wordScore=scoreList[i]/(counts[filtered_words[i]]*contextSize)
        if filtered_words[i] not in scores:
            scores[filtered_words[i]]=wordScore
        else:
            scores[filtered_words[i]]=min(scores[filtered_words[i]],wordScore)
    wordScores=list(map(list, scores.items()))
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    filteredWordScores=[]
    for word in wordScores:
        filtered=re.sub('[^a-zA-Z]', '', word[0])
        if(len(filtered)>2):
            filteredWordScores.append([filtered,word[1]])
    if collapse:
        phraseScores=collapsePhrases(tokens,scores,stops,filteredWordScores,cleaned_words)
        return phraseScores[:top]
    else:
        return filteredWordScores[:top]

In [443]:
def collapsePhrases(tokens,scores,stops,filteredWordScores,cleaned_words):
    dictionary=set()
    for word in cleaned_words:
        dictionary.add(word)
    phrases=dict()
    bagOfWords=set()
    for word in filteredWordScores:
        bagOfWords.add(word[0])
    phrase=""
    totalScore=0
    wordCount=0
    i=0
    while i in range(len(tokens)):
        word=stemmer.lemmatize(tokens[i])
        if (word in stops or tokens[i] in stops) and phrase!="":
            j=i+1
            while j<len(tokens) and stemmer.lemmatize(tokens[j]) not in dictionary:
                j+=1
            if(j<len(tokens) and stemmer.lemmatize(tokens[j]) in bagOfWords):
                for k in range(i,j):
                    phrase+=tokens[k]+" "
                i=j-1
            else:
                phrases[phrase[:len(phrase)-1]]=totalScore/wordCount
                totalScore=0
                wordCount=0
                phrase=""
        elif word in bagOfWords:
            totalScore+=scores[word]
            wordCount+=1
            phrase+=tokens[i]+" "
        else:
            if(wordCount>0):
                phrases[phrase[:len(phrase)-1]]=totalScore/wordCount
                totalScore=0
                wordCount=0
                phrase=""
        i+=1
    phraseScores=list(map(list, phrases.items()))
    phraseScores=sorted(phraseScores, key = lambda x: x[1])#,reverse=True)
    return phraseScores

In [478]:
rank("""
Japan has been trying to increase its birth rate for years, hoping that a youthful boost could offset an otherwise rapidly aging population. It's not working.

The country's health ministry announced Tuesday that the number of babies born in 2019 fell by an estimated 5.9% this year, to 864,000. It's the first time since 1899, when the government began tracking the data, that the number has dipped below 900,000, according to The Asahi Shimbun.

The decline in the absolute number of births is especially stark given that Japan's population in 1899 was about one-third of its approximately 126 million people today.

What accounts for the steep drop in births? The health ministry points to the declining numbers of people of reproductive age, as the offspring of baby boomers get older.

That joins other factors — namely the immense burden shouldered by Japanese women to do housework and childcare by themselves, and a culture that makes it difficult to both have a job outside the home and be a mother. Younger generations of Japanese women have increasingly opted to continue working, rather than get married, have children, and give up their careers.

Having children outside of wedlock is also extremely rare: just 2.3% of children were born outside of marriage in Japan, compared to about 40% in the United States. Marriage rates in Japan have halved since the early 1970s, and birth rates have declined in tandem.

At the other end of the life cycle, the numbers were also notable. Japan had its highest numbers of deaths this year since the end of World War II: nearly 1.4 million.

That means that in 2019, the country's natural population declined by more than 500,000 people — also a first, according to Asahi. The country's population is aging, and it now has the world's highest proportion of people over the age of 65.

Japan's population has declined every year since 2007. And while other countries have countered declining birth rates by permitting immigration, Japan has been slow to allow foreigners to settle there. The country granted refugee status to just 42 people last year out of the 10,000 who applied. Meanwhile, the U.S. granted asylum to more than 18,000 people in fiscal year 2019 – nearly a third of those who applied.

But Japan, hoping to counter a labor force shortage, is taking small steps to allow foreigners to at least work in the country.

On Tuesday, Japan and Pakistan signed a memorandum of cooperation to allow Pakistanis to work in Japan. According to the agreement, "specified skilled workers" must pass a test and show basic comprehension of Japanese language in order to qualify for employment in Japan.

The move follows similar agreements with other Asian countries including Vietnam, Nepal, and Cambodia.

But moving to Japan as a foreigner can be tough. As an article in The Atlantic describes, a pamphlet handed to refugees explains that "Japanese society traditionally prefers conformity and social harmony to independence and individual expression." It gave these tips on conforming: "1) Be punctual. 2) Separate garbage according to type. 3) Don't be too loud!"
""",5,True)

[['japan', 0.03323675369999061],
 ['country', 0.07103922753119651],
 ['people', 0.0815743606182791],
 ['year', 0.08619152967418943],
 ['population', 0.09935861719506127]]